In [1]:
import pandas as pd
import numpy as np
from sqlalchemy import create_engine
import pymysql
import re
from dotenv import load_dotenv
load_dotenv()
import os

In [2]:
db_connection_str = os.getenv("DB_CONNECTION_STR")

db_connection = create_engine(db_connection_str)

df = pd.read_sql('SELECT * FROM crash_db_San_Diego', con=db_connection)
df = df.replace('',np.nan)

In [3]:
df.head()

,cid,createDate,createBy,dataFrom,report_id,date_time,police_beat,address_number_primary,address_pd_primary,address_road_primary,address_sfx_primary,address_pd_intersecting,address_name_intersecting,address_sfx_intersecting,violation_section,violation_type,charge_desc,injured,killed,hit_run_lvl
0,1000000002,2020-01-28 05:34:28,David,San_Diego,180282,2018-01-01 00:30:00,124,6500,,EL CAMINO DE TEATRO,,,,,22350,VC,VIOLATION OF BASIC SPEED LAW SPEED UNSAFE FOR ...,0,0,NaN
1,1000000003,2020-01-28 05:34:28,David,San_Diego,180310,2018-01-01 01:00:00,616,500,,ROSECRANS,STREET,,,,22350,VC,VIOLATION OF BASIC SPEED LAW SPEED UNSAFE FOR ...,0,0,MISDEMEANOR
2,1000000004,2020-01-28 05:34:28,David,San_Diego,180573,2018-01-01 01:26:00,721,0,,PALM,AVENUE,,18TH,STREET,21954A,VC,PEDESTRIANS OUTSIDE CROSSWALKS,0,1,NaN
3,1000000005,2020-01-28 05:34:28,David,San_Diego,180317,2018-01-01 02:05:00,512,3100,,LOGAN,AVENUE,,,,22450(A),VC,STOP REQUIREMENTS STATE STOP LINE,0,0,MISDEMEANOR
4,1000000006,2020-01-28 05:34:28,David,San_Diego,180319,2018-01-01 02:26:00,439,6200,,DIVISION,STREET,,,,22107,VC,TURNING MOVEMENTS AND REQUIRED SIGNALS,0,0,MISDEMEANOR


In [4]:
df2 = pd.DataFrame()
df2['cid'] = df.cid
df2['city'] = df.dataFrom
df2['crash_time'] = pd.to_datetime(df.date_time)
df2['street_name'] = df.address_road_primary + df.address_sfx_primary
df2['injured_count'] = df.injured
df2['crash_date'] = pd.to_datetime(df2['crash_time'].dt.date)

In [5]:
df2.head()

,cid,city,crash_time,street_name,injured_count,crash_date
0,1000000002,San_Diego,2018-01-01 00:30:00,EL CAMINO DE TEATRO,0,2018-01-01
1,1000000003,San_Diego,2018-01-01 01:00:00,ROSECRANSSTREET,0,2018-01-01
2,1000000004,San_Diego,2018-01-01 01:26:00,PALMAVENUE,0,2018-01-01
3,1000000005,San_Diego,2018-01-01 02:05:00,LOGANAVENUE,0,2018-01-01
4,1000000006,San_Diego,2018-01-01 02:26:00,DIVISIONSTREET,0,2018-01-01


In [6]:
df2.to_sql('cleaned_crash_San_Diego',con = db_connection, if_exists="replace",index=False)

In [7]:
df2.to_sql('home_allcleanedcrashdata',con = db_connection, if_exists="append",index=False)